In [1]:

import psycopg2
from psycopg2.extras import execute_values
import pandas as pd




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 728, in start
    self.io_loop.

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 728, in start
    self.io_loop.

AttributeError: _ARRAY_API not found

In [2]:
# Definición de la función para cargar datos desde un archivo CSV a PostgreSQL
try:
    # Parámetros de conexión – AJUSTA esto a tu configuración local
    conn = psycopg2.connect(
        host="localhost",
        database="PROYECTO_FINAL_AMAZON",
        user="postgres",
        password="quique"
    )
    cursor = conn.cursor()
    print("Conexión establecida correctamente.")

except AttributeError as e:
    if '_ARRAY_API' in str(e):
        print("Advertencia: Problema interno de librerías (_ARRAY_API). Se continúa sin detener el proceso.")
    else:
        raise


Conexión establecida correctamente.


In [3]:
# Crear tabla categoria
cursor.execute("""
CREATE TABLE IF NOT EXISTS categoria (
    id SERIAL PRIMARY KEY,
    nombre TEXT NOT NULL UNIQUE
);
""")

# Crear tabla subcategoria
cursor.execute("""
CREATE TABLE IF NOT EXISTS subcategoria (
    id SERIAL PRIMARY KEY,
    nombre TEXT NOT NULL,
    categoria_id INTEGER REFERENCES categoria(id)
);
""")

# Crear tabla producto

create_table_query = """
CREATE TABLE IF NOT EXISTS producto (
    id SERIAL PRIMARY KEY,
    nombre TEXT NOT NULL,
    precio NUMERIC,
    precio_original NUMERIC,
    porcentaje_descuento NUMERIC,
    porcentaje_descuento_2 NUMERIC,
    rating NUMERIC,
    no_ratings INTEGER,
    asin TEXT,
    image TEXT,
    link TEXT,
    subcategoria_id INTEGER REFERENCES subcategoria(id)
);
"""

conn.commit()
print("Tablas creadas correctamente.")


Tablas creadas correctamente.


In [4]:
# Ruta al archivo CSV limpio 
df = pd.read_csv("c:/Users/User/Desktop/hackio/proyectos/Proyecto_Final/Proyecto_final/data/data_procesed/amazon_data_clean_eda.csv")

# Confirmamos que se cargó correctamente
print("DataFrame cargado con shape:", df.shape)
df.head()


DataFrame cargado con shape: (738060, 12)


,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,discount_percent,asin,discount_percentage
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,2255,32999.0,58990.0,44.0,B0BRKXTSBT,44.060010
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,2948,46490.0,75990.0,39.0,B0BQ3MXML8,38.820897
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,1206,34490.0,61990.0,44.0,B0BPYN9JGF,44.361994
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69,37990.0,68990.0,45.0,B0BQ3MJ1TG,44.934048
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630,34490.0,67790.0,49.0,B0B67RLLJC,49.122289


In [5]:
# Después de cargar el DataFrame (df), aplica esta corrección:
df['discount_price'] = df['discount_price'] / 100
df['actual_price']   = df['actual_price']   / 100

# Opcional: si también quieres ver que quedó bien:
print(df[['name', 'discount_price', 'actual_price']].head())

                                                name  discount_price  \
0  Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...          329.99   
1  LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...          464.90   
2  LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...          344.90   
3  LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...          379.90   
4  Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...          344.90   

   actual_price  
0         589.9  
1         759.9  
2         619.9  
3         689.9  
4         677.9  


In [6]:
df

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,discount_percent,asin,discount_percentage
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,2255,329.99,589.90,44.0,B0BRKXTSBT,44.060010
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,2948,464.90,759.90,39.0,B0BQ3MXML8,38.820897
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,1206,344.90,619.90,44.0,B0BPYN9JGF,44.361994
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69,379.90,689.90,45.0,B0BQ3MJ1TG,44.934048
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630,344.90,677.90,49.0,B0B67RLLJC,49.122289
...,...,...,...,...,...,...,...,...,...,...,...,...
738055,BE SAVAGE Pink I Purple You Hoodie Suitable fo...,sports & fitness,Yoga,https://m.media-amazon.com/images/I/61voQ68VuT...,https://www.amazon.in/Be-savage-Purple-Hoodie-...,4.6,6,7.11,9.99,29.0,B08XZM7KSC,28.828829
738056,Adidas Regular Fit Men's Track Tops,sports & fitness,Yoga,https://m.media-amazon.com/images/I/71tHAR9pIY...,https://www.amazon.in/Adidas-Regular-Mens-Trac...,3.2,9,34.49,45.99,25.0,B08X7X2YNB,25.005436
738057,Redwolf Noice Toit Smort - Hoodie (Black),sports & fitness,Yoga,https://m.media-amazon.com/images/I/41pKrMZ5lQ...,https://www.amazon.in/Redwolf-Noice-Smort-Cott...,2.0,2,11.99,19.99,40.0,B08X1MB7LR,40.020010
738058,Redwolf Schrute Farms B&B - Hoodie (Navy Blue),sports & fitness,Yoga,https://m.media-amazon.com/images/I/41n9u+zNSc...,https://www.amazon.in/Redwolf-Schrute-Farms-Ho...,4.0,1,11.99,19.99,40.0,B08X1HGKPZ,40.020010


In [7]:
# Columnas que deberían ser numéricas
num_cols = [
    'discount_price',
    'actual_price',
    'discount_percent',
    'discount_percentage',
    'ratings',
    'no_of_ratings'
]

# Función para detectar celdas que aún sean str
def rows_con_texto(df, cols):
    mask_global = pd.Series(False, index=df.index)
    for col in cols:
        mask_col = df[col].apply(lambda x: isinstance(x, str))
        if mask_col.any():
            print(f"⚠️  Valores no numéricos en «{col}»:")
            print(df.loc[mask_col, ['name', col]].head())
        mask_global |= mask_col
    return df[mask_global]

# Comprueba todo el DataFrame
filas_erroneas = rows_con_texto(df, num_cols)
print(f"Filas con texto en columnas numéricas: {len(filas_erroneas)}")


⚠️  Valores no numéricos en «ratings»:
                                                name ratings
0  Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...     4.2
1  LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...     4.2
2  LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...     4.2
3  LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...     4.0
4  Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...     4.1
Filas con texto en columnas numéricas: 738060


In [8]:
df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')


In [9]:
categorias = df['main_category'].dropna().drop_duplicates()

for cat in categorias:
    cursor.execute("""
        INSERT INTO categoria (nombre)
        VALUES (%s)
        ON CONFLICT (nombre) DO NOTHING;
    """, (cat,))
conn.commit()
print("Categorías insertadas.")


Categorías insertadas.


In [10]:
# Creamos diccionario de categorías
cursor.execute("SELECT id, nombre FROM categoria")
categoria_map = {nombre: id for id, nombre in cursor.fetchall()}

In [11]:
# Insertar subcategorías
subcategorias = df[['main_category', 'sub_category']].drop_duplicates()

for _, row in subcategorias.iterrows():
    nombre_sub = row['sub_category']
    nombre_cat = row['main_category']
    categoria_id = categoria_map.get(nombre_cat)

    if categoria_id:
        cursor.execute("""
            INSERT INTO subcategoria (nombre, categoria_id)
            VALUES (%s, %s)
            ON CONFLICT DO NOTHING;
        """, (nombre_sub, categoria_id))
conn.commit()
print("Subcategorías insertadas.")


Subcategorías insertadas.


In [12]:
# Cremos diccionario para mapear subcategorías a sus IDs
cursor.execute("SELECT id, nombre FROM subcategoria")
subcategoria_map = {nombre: id for id, nombre in cursor.fetchall()}

In [13]:
# Cerramos la conexión
cursor.close()
conn.close()
print("Conexión cerrada.")

Conexión cerrada.


In [14]:
# Limpiar la columna 'ratings' para que contenga solo valores numéricos
def limpiar_rating(valor):
    try:
        return float(valor)
    except:
        return None  # o np.nan si usas numpy

df['ratings'] = df['ratings'].apply(limpiar_rating)


In [15]:
# 🔧 Conexión a la base de datos
conn = psycopg2.connect(
    host="localhost",
    database="PROYECTO_FINAL_AMAZON",
    user="postgres",
    password="quique"
)
cursor = conn.cursor()
print("Conexión establecida correctamente.")

# Volver a crear el diccionario subcategoria_map
cursor.execute("SELECT id, nombre FROM subcategoria")
subcategoria_map = {nombre: id for id, nombre in cursor.fetchall()}

# 🧱 1. Borramos y recreamos la tabla 'producto'
cursor.execute("DROP TABLE IF EXISTS producto;")
conn.commit()

create_table_query = """
CREATE TABLE producto (
    id SERIAL PRIMARY KEY,
    nombre TEXT NOT NULL,
    precio_con_descuento NUMERIC,
    precio_original NUMERIC,
    porcentaje_descuento NUMERIC,
    rating NUMERIC,
    no_ratings INTEGER,
    asin TEXT,
    image TEXT,
    link TEXT,
    subcategoria_id INTEGER REFERENCES subcategoria(id)
);
"""
cursor.execute(create_table_query)
conn.commit()
print("Tabla 'producto' creada desde cero.")

# 📦 2. Preparar datos para inserción
productos_data = []
for _, row in df.iterrows():
    subcat_id = subcategoria_map.get(row['sub_category'])
    if subcat_id:
        productos_data.append((
            row.get('name'),
            row.get('discount_price'),
            row.get('actual_price'),
            row.get('discount_percentage'),
            row.get('ratings'),
            row.get('no_of_ratings'),
            row.get('asin'),
            row.get('image'),
            row.get('link'),
            subcat_id
        ))

# 📤 3. Insertar datos
insert_query = """
INSERT INTO producto (
    nombre, precio_con_descuento, precio_original,porcentaje_descuento,
    rating, no_ratings, asin, image, link, subcategoria_id
) VALUES %s
"""

try:
    execute_values(cursor, insert_query, productos_data)
    conn.commit()
    print("Inserción masiva completada correctamente.")
except Exception as e:
    conn.rollback()
    print("❌ Error al insertar productos:", e)



Conexión establecida correctamente.
Tabla 'producto' creada desde cero.
Inserción masiva completada correctamente.


In [16]:
# ✅ 4. Cerrar conexión
cursor.close()
conn.close()